# Dataset Analysis Demo

This notebook demonstrates how to use the high-level analysis tooling from OG-MARL to inspect datasets.

In [1]:
from og_marl.vault_utils.download_vault import *

In [2]:
print_dataset_options()

{'cfcql': {'smac_v1': {'2s3z': {...},
                       '3s_vs_5z': {...},
                       '5m_vs_6m': {...},
                       '6h_vs_8z': {...}}},
 'og_marl': {'mamujoco': {'2halfcheetah': {...}},
             'smac_v1': {'2s3z': {...},
                         '3m': {...},
                         '3s5z_vs_3s6z': {...},
                         '5m_vs_6m': {...},
                         '8m': {...}},
             'smac_v2': {'terran_5_vs_5': {...}, 'zerg_5_vs_5': {...}}}}


In [6]:
download_and_unzip_vault('og_marl','smac_v1','didi',dataset_download_url="dipidipidababa")

UnboundLocalError: local variable 'response' referenced before assignment